# LSTM + Attention Notebook

This is a notebook to implement the LSTM model with Attention features within, will also be using the same data the LSTM Notebook will, which is the RNN's Preprocessing Output.